In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import deque
import datetime as dt
import mplfinance 
from renkodf import Renko
from scipy.signal import lfilter
import fx


In [3]:
# collect the actual data from csv file
filename2023 = "C:/Users/WilliamFetzner/Documents/Trading/DAT_XLSX_EURUSD_M1_2023.xlsx"
filename2022 = "C:/Users/WilliamFetzner/Documents/Trading/DAT_XLSX_EURUSD_M1_2022.xlsx"

df_2023 = pd.read_excel(filename2023, header=None, names=['datetime', 'open', 'high', 'low', 'close', 'volume'])
df_2022 = pd.read_excel(filename2022, header=None, names=['datetime', 'open', 'high', 'low', 'close', 'volume'])

c:\Users\WilliamFetzner\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\WilliamFetzner\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
df_2022 = fx.prep_data(df_2022)
df_2023 = fx.prep_data(df_2023)

# Parameters

In [8]:
# Renko variable
initial_brick_size = 0.000325
# create a list of possible brick sizes a max at 0.001 and min at 0.00001 and each step is 0.00001
brick_size_list = np.arange(0.00001, 0.00101, 0.00001)

# psar variables
start = 0.02
increment = 0.02
increment_list = np.arange(0.001, 0.2001, 0.0001)
maximum = 0.2
max_list = np.arange(0.01, 0.501, 0.001)

# impulse variables
lengthMA = 34
ma_list = np.arange(10, 100, 1)
lengthSignal = 9
signal_list = np.arange(1, 51, 1)

# Lot Size
lot_size = 10
lot_sizes_list = np.arange(1, 21, 1)
per_lot = 100000

# Commissions
nova_commission = lot_size * -3
msolutions_commission = lot_size * -5

# starting balance
balance = 200000

# base currency rate
base_currency_rate = 0.045 # EUR
quote_currency_rate = 0.055 # USD

In [9]:
brick_sizes

array([1.0e-05, 2.0e-05, 3.0e-05, 4.0e-05, 5.0e-05, 6.0e-05, 7.0e-05,
       8.0e-05, 9.0e-05, 1.0e-04, 1.1e-04, 1.2e-04, 1.3e-04, 1.4e-04,
       1.5e-04, 1.6e-04, 1.7e-04, 1.8e-04, 1.9e-04, 2.0e-04, 2.1e-04,
       2.2e-04, 2.3e-04, 2.4e-04, 2.5e-04, 2.6e-04, 2.7e-04, 2.8e-04,
       2.9e-04, 3.0e-04, 3.1e-04, 3.2e-04, 3.3e-04, 3.4e-04, 3.5e-04,
       3.6e-04, 3.7e-04, 3.8e-04, 3.9e-04, 4.0e-04, 4.1e-04, 4.2e-04,
       4.3e-04, 4.4e-04, 4.5e-04, 4.6e-04, 4.7e-04, 4.8e-04, 4.9e-04,
       5.0e-04, 5.1e-04, 5.2e-04, 5.3e-04, 5.4e-04, 5.5e-04, 5.6e-04,
       5.7e-04, 5.8e-04, 5.9e-04, 6.0e-04, 6.1e-04, 6.2e-04, 6.3e-04,
       6.4e-04, 6.5e-04, 6.6e-04, 6.7e-04, 6.8e-04, 6.9e-04, 7.0e-04,
       7.1e-04, 7.2e-04, 7.3e-04, 7.4e-04, 7.5e-04, 7.6e-04, 7.7e-04,
       7.8e-04, 7.9e-04, 8.0e-04, 8.1e-04, 8.2e-04, 8.3e-04, 8.4e-04,
       8.5e-04, 8.6e-04, 8.7e-04, 8.8e-04, 8.9e-04, 9.0e-04, 9.1e-04,
       9.2e-04, 9.3e-04, 9.4e-04, 9.5e-04, 9.6e-04, 9.7e-04, 9.8e-04,
       9.9e-04, 1.0e

# Optimization

In [ ]:
# create a renko chart from the df_2022 dataframe
r_full_2022 = Renko(df_2022, brick_size=brick_size)
# create a new dataframe from the renko features
renko_full_data_2022 = r_full_2022.renko_df()
# rename 'volume' to 'nbars'
renko_full_data_2022 = renko_full_data_2022.rename(columns={'volume': 'nbars'})
r_2022_w_psar = fx.psar_from_data(renko_full_data_2022)
r_2023_w_psar = fx.psar_from_data(renko_full_data_2023)